# Hashtags with geo/time information for ploting

This ipython notebook is used to add hashtag information to geo/time dataframe.

In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import json
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import time
import pickle
import os

In [3]:
with open("data/twitter-swisscom/geo_info_total.pickle", "rb") as f:
    geo_info_total = pickle.load(f)

In [4]:
with open("data/twitter-swisscom/twitter_tag_20140102.pickle", "rb") as f:
    twitter_tag = pickle.load(f)

In [5]:
from collections import defaultdict
def convert_upper_to_lower_in_dict(d):
    r = defaultdict(int)
    for k, v in d.items():
        r[k.lower()] += v
        
    return r

In [6]:
twitter_tag['tag_freq'] = twitter_tag['tag_freq'].apply(lambda x: convert_upper_to_lower_in_dict(x))

In [7]:
from collections import Counter
all_tags_count = sum((Counter(dict(x)) for x in twitter_tag['tag_freq']),Counter())

We only choose the top hashtags (expect empty hashtags)

In [9]:
top_hash_tags = sorted(all_tags_count.items(), key=lambda t: -t[1])[:13]
del top_hash_tags[7]

only keeps these items

In [10]:
def filter_top_hash_tags(d):
    r = {k: v for k, v in d.items() if k in dict(top_hash_tags).keys()}
    return r # sorted(r.items(), key=lambda t: -t[1])

In [11]:
twitter_tag['tag_freq'] = twitter_tag['tag_freq'].apply(lambda x: filter_top_hash_tags(x))

In [12]:
processed_twitter_tag = twitter_tag[twitter_tag['tag_freq'].apply(lambda x: len(x) > 0)]

In [13]:
pivoted_tw_tags = processed_twitter_tag.reset_index('hour').pivot( values='tag_freq', columns='hour')

In [14]:
pivoted_tw_tags = pd.merge(pivoted_tw_tags.reset_index(), geo_info_total, on='placeId', how='left')

In [16]:
pivoted_tw_tags[(pivoted_tw_tags.placeId == 'ff8ee52edf03e6bd') | 
                (pivoted_tw_tags.placeId == "fc58a3476b706856")].to_csv("web/tw_tags.csv")

In [17]:
d = pivoted_tw_tags[(pivoted_tw_tags.placeId == 'ff8ee52edf03e6bd') | (pivoted_tw_tags.placeId == "fc58a3476b706856")]
d

,placeId,0,1,3,4,7,8,9,11,13,...,16,17,18,19,20,21,22,23,placeLatitude,placeLongitude
108,fc58a3476b706856,None,None,None,None,None,None,None,None,None,...,"{'#sherlock': 1, '#newyearrocks': 68}","{'#awesome': 1, '#sherlock': 1, '#amazing': 1,...","{'#awesome': 1, '#sherlock': 3, '#winterclassi...","{'#justloveit': 1, '#sherlocklives': 1, '#newy...","{'#awesome': 2, '#2014': 1, '#newyearrocks': 3...",None,{'#justloveit': 1},None,47.7258,8.63805
109,ff8ee52edf03e6bd,None,None,None,None,None,None,None,None,{'#justloveit': 1},...,"{'#happynewyear': 1, '#hgtvnewyear': 1, '#just...","{'#winterclassic': 1, '#sherlocklives': 1, '#n...","{'#sherlock': 2, '#hgtvnewyear': 1, '#wintercl...","{'#2014': 1, '#winterclassic': 1, '#justloveit...","{'#awesome': 1, '#happynewyear': 1, '#wintercl...",{'#justloveit': 2},None,{'#justloveit': 1},46.0509,9.19546


In [19]:
pivoted_tw_tags.to_csv("web/tw_tags.csv")